In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import click
from pathlib import Path
import preprocessor as p
import re

# text Manipulation for TWIVER CMU

The next few lines are for our manipulation and not part of the examples for BERT below

In [2]:
file_day = 'ukraine_day_processed.csv'
turk_file_day = 'ukraine_day_processed_turked.csv'

file_week = 'ukraine_week_processed.csv'
turk_file_week = 'ukraine_week_processed_turked.csv'

file_month = 'ukraine_month_processed.csv'
turk_file_month= 'ukraine_month_processed_turked.csv'

path = Path.cwd()
text_Dir = path.joinpath('ukraine_processed')
turk_Dir = path.joinpath('ukraine_turked')
by_hand_Dir = turk_Dir.joinpath('ukraine_turked_byhand')

file_Dir_day = text_Dir.joinpath(file_day)
turkFile_Dir_day = by_hand_Dir.joinpath(turk_file_day)

file_Dir_week = text_Dir.joinpath(file_week)
turkFile_Dir_week = by_hand_Dir.joinpath(turk_file_week)

file_Dir_month = text_Dir.joinpath(file_month)
turkFile_Dir_month = by_hand_Dir.joinpath(turk_file_month)
#print(text_Dir)
tweets_processed_day = pd.read_csv(file_Dir_day, dtype='str' , names=['id', 'tweet'])
tweets_turked_day = pd.read_csv(turkFile_Dir_day, dtype='str' , names=['non-id', 'tweet', 'rating'])

tweets_processed_week = pd.read_csv(file_Dir_week, dtype='str' , names=['id', 'tweet'])
tweets_turked_week = pd.read_csv(turkFile_Dir_week, dtype='str' , names=['non-id', 'tweet', 'rating'])

tweets_processed_month = pd.read_csv(file_Dir_month, dtype='str' , names=['id', 'tweet'])
tweets_turked_month = pd.read_csv(turkFile_Dir_month, dtype='str' , names=['non-id', 'tweet', 'rating'])


In [3]:
day_merge = pd.merge(tweets_processed_day, tweets_turked_day, 'right', 'tweet')
day_merge = day_merge.drop_duplicates(subset=['tweet'])
day_merge = day_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
day_merge = day_merge[day_merge.rating.isin(values) ==True ]
day_outpath = turk_Dir.joinpath('ukraine_day_turked')
print(day_merge.head(10))
print(len(day_merge.drop_duplicates()))
day_merge.to_csv(day_outpath, header=False, index=False)

                     id                                              tweet  \
0   1496273443754496003  @SocComdeeznuts @theschwasound @LobbyDob No it...   
1   1496273551048986627  @theRedacted0ne &gt;implying itll happen\nruss...   
2   1496273438381686790  Nothing but lies.  Ukraine claims there are no...   
3   1496273417372373000  Just because anonymous US officials have been ...   
9   1496273394383441926  @JackPosobiec We are being played.  #Ukraine #...   
10  1496273370907828224  There is worry that #Russia will invade #Ukrai...   
11  1496273362737373186  @freedomrideblog I didn’t realize the us put 1...   
12  1496273357117206528  Putin Invades Ukraine (But Not Really) https:/...   
13  1496273331703599106  @NBCNews Stop the lies about Putin and Russia....   
14  1496273597719056386  @Denis_Buchholz @Cinderoak This is when Putin'...   

   rating  
0      UC  
1      DN  
2      DN  
3      DN  
9      DN  
10     DN  
11     DN  
12     DN  
13     DN  
14     DY  
1511


In [4]:
day_merge['TimeFrame'] = 'day'
day_merge.head(10)

,id,tweet,rating,TimeFrame
0,1496273443754496003,@SocComdeeznuts @theschwasound @LobbyDob No it...,UC,day
1,1496273551048986627,@theRedacted0ne &gt;implying itll happen\nruss...,DN,day
2,1496273438381686790,Nothing but lies. Ukraine claims there are no...,DN,day
3,1496273417372373000,Just because anonymous US officials have been ...,DN,day
9,1496273394383441926,@JackPosobiec We are being played. #Ukraine #...,DN,day
10,1496273370907828224,There is worry that #Russia will invade #Ukrai...,DN,day
11,1496273362737373186,@freedomrideblog I didn’t realize the us put 1...,DN,day
12,1496273357117206528,Putin Invades Ukraine (But Not Really) https:/...,DN,day
13,1496273331703599106,@NBCNews Stop the lies about Putin and Russia....,DN,day
14,1496273597719056386,@Denis_Buchholz @Cinderoak This is when Putin'...,DY,day


In [35]:
#To fix after turking uncoment comments below
week_merge = pd.merge(tweets_processed_week, tweets_turked_week, 'right', 'tweet')
week_merge = week_merge.drop_duplicates(subset=['tweet'])
week_merge = week_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
week_merge = week_merge[week_merge.rating.isin(values) ==True ]
week_outpath = turk_Dir.joinpath('ukraine_week_turked')
#week_returk_outpath = turk_Dir.joinpath('ukraine_week_returk')
print(week_merge.head(10))
print(len(week_merge))
week_merge.to_csv(week_outpath, header=False, index=False)
#week_merge.to_csv(week_returk_outpath, header=False, index=False)

                      id                                              tweet  \
0    1494099270399627266  @POTUS you and nato are the ones violating Rus...   
1    1494099269858603008  BREAKING: The Biden adminsitration says within...   
27   1494099268176527360  #UPDATE: #Russia has been building bridges and...   
28   1494099264636481540  If Russia launches an invasion of Ukraine, mil...   
29   1494099264452059142  The Biden administration telling US citizens t...   
30   1494099263567061003  A senior Central African Republic military off...   
40   1494099262526873602  NATO chief, US, and open source show no pullba...   
48   1494099259116904448  Russia has added about 7,000 troops along Ukra...   
147  1494099258773065732  As much as I loathe Putin, he has shown up Bid...   
148  1494099257388896258  Maria Zakharova, the spokeswoman for Russia's ...   

    rating  
0       UC  
1       PY  
27      PY  
28      PY  
29      PY  
30      PY  
40      PY  
48      PY  
147     UC  


In [36]:
#To fix after turking uncoment comments below
month_merge = pd.merge(tweets_processed_month, tweets_turked_month, 'right', 'tweet')
month_merge = month_merge.drop_duplicates(subset=['tweet'])
month_merge = month_merge.loc[:, ['id', 'tweet', 'rating']]
values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
month_merge = month_merge[month_merge.rating.isin(values) ==True ]
month_outpath = turk_Dir.joinpath('ukraine_month_turked')
#month_returk_outpath = turk_Dir.joinpath('ukraine_month_returk')
print(month_merge.head(10))
print(len(month_merge))
month_merge.to_csv(month_outpath, header=False, index=False)
#month_merge.to_csv(month_returk_outpath, header=False, index=False)

                      id                                              tweet  \
1    1485039574556438530  @disclosetv Poppycock. Nothing to see here. Me...   
2    1485039574288150530  In a bid to continue the attacks, @AbiyAhmedAl...   
3    1485039574027886593  BREAKING NEWS\n\nPUTIN PREPARING TO INSTALL PR...   
21   1485039571796692999  Hope these planes in Ukraine have better wheel...   
83   1485039570576023555  LONDON (AP) — British government accuses Russi...   
87   1485039570169303043  United States is creating this crisis by encou...   
88   1485039569825374209  Russia aims to replace Ukraine government with...   
93   1485039567912816640  What is Joe Biden’s plan to stop Russia from i...   
122  1485039566285422606  I think we should start paying far more attent...   
127  1485039565366706178  THERE SHOULD BE NO AMERICAN INVOLVEMENT IN UKR...   

    rating  
1       DY  
2       DY  
3       DY  
21      DY  
83      DY  
87      DY  
88      DY  
93      DY  
122     DY  


In [37]:
week_merge['TimeFrame'] = 'week'
week_merge.head(10)

,id,tweet,rating,TimeFrame
0,1494099270399627266,@POTUS you and nato are the ones violating Rus...,UC,week
1,1494099269858603008,BREAKING: The Biden adminsitration says within...,PY,week
27,1494099268176527360,#UPDATE: #Russia has been building bridges and...,PY,week
28,1494099264636481540,"If Russia launches an invasion of Ukraine, mil...",PY,week
29,1494099264452059142,The Biden administration telling US citizens t...,PY,week
30,1494099263567061003,A senior Central African Republic military off...,PY,week
40,1494099262526873602,"NATO chief, US, and open source show no pullba...",PY,week
48,1494099259116904448,"Russia has added about 7,000 troops along Ukra...",PY,week
147,1494099258773065732,"As much as I loathe Putin, he has shown up Bid...",UC,week
148,1494099257388896258,"Maria Zakharova, the spokeswoman for Russia's ...",PN,week


In [38]:
month_merge['TimeFrame'] = 'month'
month_merge.head(10)

,id,tweet,rating,TimeFrame
1,1485039574556438530,@disclosetv Poppycock. Nothing to see here. Me...,DY,month
2,1485039574288150530,"In a bid to continue the attacks, @AbiyAhmedAl...",DY,month
3,1485039574027886593,BREAKING NEWS\n\nPUTIN PREPARING TO INSTALL PR...,DY,month
21,1485039571796692999,Hope these planes in Ukraine have better wheel...,DY,month
83,1485039570576023555,LONDON (AP) — British government accuses Russi...,DY,month
87,1485039570169303043,United States is creating this crisis by encou...,DY,month
88,1485039569825374209,Russia aims to replace Ukraine government with...,DY,month
93,1485039567912816640,What is Joe Biden’s plan to stop Russia from i...,DY,month
122,1485039566285422606,I think we should start paying far more attent...,DY,month
127,1485039565366706178,THERE SHOULD BE NO AMERICAN INVOLVEMENT IN UKR...,PY,month


In [39]:
total_merge = pd.concat([day_merge, week_merge, month_merge])
total_merge.head(10)
total_outpath = text_Dir.joinpath('ukraine_turked_total')


In [40]:
def preprocess_text(text):
 #Removes Numbers
 text = text.replace('\d+', '')
 str = text.lower()
 str = re.sub("'", "", str) # to avoid removing contractions in english
 str = re.sub("@[A-Za-z0-9_]+","", str)
 str = re.sub("#[A-Za-z0-9_]+","", str)
 str = re.sub(r'http\S+', '', str)
 str = re.sub('[()!?]', ' ', str)
 str = re.sub('\[.*?\]',' ', str)
 str = re.sub("[^a-z0-9]"," ", str)
 return str

In [41]:
total_merge['tweet'] = total_merge['tweet'].map(lambda x: p.clean(x))
total_merge['tweet'] = total_merge['tweet'].map(lambda x: preprocess_text(x))

print(len(total_merge))
total_merge.head(10)

2931


,id,tweet,rating,TimeFrame
0,1496273443754496003,no its the russia is communist right i lear...,UC,day
1,1496273551048986627,gt implying itll happen nrussia invading ukra...,DN,day
2,1496273438381686790,nothing but lies ukraine claims there are no ...,DN,day
3,1496273417372373000,just because anonymous us officials have been ...,DN,day
9,1496273394383441926,we are being played,DN,day
10,1496273370907828224,there is worry that will invade has denied i...,DN,day
11,1496273362737373186,i didnt realize the us put k troops on the bor...,DN,day
12,1496273357117206528,putin invades ukraine but not really via,DN,day
13,1496273331703599106,stop the lies about putin and russia this is ...,DN,day
14,1496273597719056386,this is when putins invasion of ukraine began ...,DY,day


In [42]:
total_merge.to_csv(total_outpath, header=False, index=False)

In [43]:
#values = ['DY', 'PY' , 'UC' , 'PN', 'DN'] 
#week_merge = week_merge[week_merge.rating.isin(values) ==True ]
positive_total = total_merge[total_merge.rating.isin(['PY','DY'])== True]
print('Positive total: ', len(positive_total))
neutral_total = total_merge[total_merge.rating.isin(['UC'])== True]
print('Neutral total: ', len(neutral_total))
negative_total = total_merge[total_merge.rating.isin(['PN','DN'])== True]
print('Negative total: ', len(negative_total))

Positive total:  1885
Neutral total:  820
Negative total:  226


In [44]:
#Split the data in to train/ dev/ test
total_train = total_merge.sample(frac = .8)
total_test = total_merge.drop(total_train.index)
total_dev = total_test.sample(frac = .5)
total_test = total_test.drop(total_dev.index)

In [45]:
print('Total Train size: ', len(total_train))
positive_train = total_train[total_train.rating.isin(['PY','DY'])== True]
print('Positive total: ', len(positive_train))
neutral_train = total_train[total_train.rating.isin(['UC'])== True]
print('Neutral total: ', len(neutral_train))
negative_train = total_train[total_train.rating.isin(['PN','DN'])== True]
print('Negative total: ', len(negative_train))
total_train.head(10)

Total Train size:  2345
Positive total:  1528
Neutral total:  628
Negative total:  189


,id,tweet,rating,TimeFrame
799,1494099203089309701,s stotenberg russian troops are moving into...,PY,week
949,1496273263512756225,well you can see quite clearly from this map w...,DY,day
1962,1496273129882144781,expect putin to take this seriously nah,UC,day
3364,1496273069601660931,russia wont take this as response either,PY,day
1152,1496273099989426176,we must complete our recovery from the embers...,DY,day
30208,1494098061311762439,breaking nato chief says that russia is conti...,PY,week
986,1496273238963458048,the republican party is consumed in a cultish ...,DY,day
38289,1485038298594643970,the british government said that the kremlin w...,PY,month
3426,1496273034071715840,im still confused the us isnt going to use ...,PY,day
17137,1494098503659835393,us says russia has added troops along ukraine ...,PY,week


In [46]:
print('Total Dev size: ', len(total_dev))
positive_dev = total_dev[total_dev.rating.isin(['PY','DY'])== True]
print('Positive dev: ', len(positive_dev))
neutral_dev = total_dev[total_dev.rating.isin(['UC'])== True]
print('Neutral dev: ', len(neutral_dev))
negative_dev = total_dev[total_dev.rating.isin(['PN','DN'])== True]
print('Negative dev: ', len(negative_dev))
total_dev.head(10)

Total Dev size:  276
Positive dev:  175
Neutral dev:  87
Negative dev:  14


,id,tweet,rating,TimeFrame
2014,1496273074555179015,jennifer mccarthy rubin is gonna start takin...,UC,day
11747,1485039209194864640,cnn the us embassy in kiev has asked the stat...,PY,month
116528,1485035818809012228,the british government on saturday accused rus...,PY,month
3173,1496273197418745860,if youre a sff fan concerned about ukraine th...,PY,day
2860,1496273502605029376,why putin doesnt want nato in ukraine,PY,day
3010,1496273317803794434,biden says will impose sanctions on russias e...,PY,day
10778,1494098748326166537,nope most sane people realize that americas t...,UC,week
121445,1485035694246359042,ordered us embassy in ukraine to leave,PY,month
33199,1494097978394656769,vladimir putins forces could keep up war games...,UC,week
25535,1485038711301677059,if true the plan has its challenges these pr...,PY,month


In [47]:
print('Total Test size: ', len(total_test))
positive_test = total_test[total_test.rating.isin(['PY','DY'])== True]
print('Positive test: ', len(positive_test))
neutral_test = total_test[total_test.rating.isin(['UC'])== True]
print('Neutral test: ', len(neutral_test))
negative_test = total_test[total_test.rating.isin(['PN','DN'])== True]
print('Negative test: ', len(negative_test))
total_test.head(10)

Total Test size:  277
Positive test:  164
Neutral test:  95
Negative test:  18


,id,tweet,rating,TimeFrame
250,1496273572553232385,whole world fuck russia,DY,day
494,1496273501443215360,kenyas ambassador to the un security council s...,DY,day
532,1496273489803755522,she also supports putins invasion of ukraine n,DY,day
548,1496273486200909824,president biden who in the lords name does p...,DY,day
600,1496273467674697729,ukraine will fall,DY,day
601,1496273467548876807,as vladimir putin invades ukraine i remember...,DY,day
614,1496273461517467650,ive been saying for quite a while nobody wins ...,DY,day
621,1496273457310572544,first isis came back now putin is dreaming ab...,DY,day
631,1496273447563018244,this has very little if anything to do with na...,DY,day
670,1496273427573125121,president ukraine is not just a neighbouring...,DY,day


# Bert Example

The downloaded textset is in tgz format. We can open it using `tarfile.open()` and then extract the csv files using the `extractall()` method:

Let's look at the the first 5 rows of the train and test textsets to understand the text we are dealing with:

We can see that in our textset a label of 1 means the review is bad while a label of 2 means the review is good.

Let's change this to a more standard pattern — 0 and 1 labels. Let's have a label 0 for the bad review and a label 1 for the good review:

In [48]:
total_train['rating'] = total_train['rating'] .replace(['DY', 'PY', 'UC' , 'PN', 'DN'] , [0, 0, 1,2, 2] )
total_dev['rating'] = total_dev['rating'] .replace(['DY', 'PY' , 'UC' , 'PN', 'DN'] , [0, 0, 1, 2, 2] )
total_test['rating'] = total_test['rating'] .replace(['DY', 'PY' , 'UC' , 'PN', 'DN'] , [0, 0, 1, 2, 2] )

In [49]:
total_train.head()

,id,tweet,rating,TimeFrame
799,1494099203089309701,s stotenberg russian troops are moving into...,0,week
949,1496273263512756225,well you can see quite clearly from this map w...,0,day
1962,1496273129882144781,expect putin to take this seriously nah,1,day
3364,1496273069601660931,russia wont take this as response either,0,day
1152,1496273099989426176,we must complete our recovery from the embers...,0,day


In [50]:
total_test.head()

,id,tweet,rating,TimeFrame
250,1496273572553232385,whole world fuck russia,0,day
494,1496273501443215360,kenyas ambassador to the un security council s...,0,day
532,1496273489803755522,she also supports putins invasion of ukraine n,0,day
548,1496273486200909824,president biden who in the lords name does p...,0,day
600,1496273467674697729,ukraine will fall,0,day


In [51]:
total_train['let'] = 'a'
total_train.head()

,id,tweet,rating,TimeFrame,let
799,1494099203089309701,s stotenberg russian troops are moving into...,0,week,a
949,1496273263512756225,well you can see quite clearly from this map w...,0,day,a
1962,1496273129882144781,expect putin to take this seriously nah,1,day,a
3364,1496273069601660931,russia wont take this as response either,0,day,a
1152,1496273099989426176,we must complete our recovery from the embers...,0,day,a


In [52]:
total_dev['let'] = 'a'
total_dev.head()

,id,tweet,rating,TimeFrame,let
2014,1496273074555179015,jennifer mccarthy rubin is gonna start takin...,1,day,a
11747,1485039209194864640,cnn the us embassy in kiev has asked the stat...,0,month,a
116528,1485035818809012228,the british government on saturday accused rus...,0,month,a
3173,1496273197418745860,if youre a sff fan concerned about ukraine th...,0,day,a
2860,1496273502605029376,why putin doesnt want nato in ukraine,0,day,a


In [53]:
total_train_final = total_train.loc[:, ['id','rating','let','tweet']]
total_train_final.head()

,id,rating,let,tweet
799,1494099203089309701,0,a,s stotenberg russian troops are moving into...
949,1496273263512756225,0,a,well you can see quite clearly from this map w...
1962,1496273129882144781,1,a,expect putin to take this seriously nah
3364,1496273069601660931,0,a,russia wont take this as response either
1152,1496273099989426176,0,a,we must complete our recovery from the embers...


In [54]:
total_dev_final = total_dev.loc[:, ['id','rating','let','tweet']]
#total_test['id']:range(len(total_test))
total_test_final = total_test.loc[:, ['id','tweet']]

In [55]:
total_train_final.to_csv('ukraine_train.tsv', sep='\t', index=False, header=False)
total_dev_final.to_csv('ukraine_dev.tsv', sep='\t', index=False, header=False)
total_test_final.to_csv('ukraine_test.tsv', sep='\t', index=False, header=False)

### Making things BERT friendly
1. First let's make the text compliant with BERT:

    * Column 0: An ID for the row. (Required both for train and test text.)
    * Column 1: The class label for the row. (Required only for train text.)
    * Column 2: A column of the same letter for all rows — this is a throw-away column that we need to include because BERT expects it. (Required only for train text.)
    * Column 3: The text examples we want to classify. (Required both for train and test text.)

2. We need to split the files into the format expected by BERT: BERT comes with text loading classes that expects two files called train and dev for training. In addition, BERT’s text loading classes can also use a test file but it expects the test file to be unlabelled.


3. Once the text is in the correct format, we need to save the files as .tsv (BERT doesn't take .csv as input.)